In [155]:
import pandas as pd
import numpy as np

In [156]:
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [157]:
df1 = df1.drop(['PID','Garage_Yr_Blt'], axis=1)
df2 = df2.drop(['PID','Garage_Yr_Blt'], axis=1)
df1

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude,Sale_Price
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,None,0,5,2010,WD,Normal,-93.619754,42.054035,215000
1,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,-93.619387,42.052659,172000
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,None,0,4,2010,WD,Normal,-93.617320,42.051245,244000
3,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,None,0,3,2010,WD,Normal,-93.638933,42.060899,189900
4,One_Story_PUD_1946_and_Newer,Residential_Low_Density,43,5005,Pave,No_Alley_Access,Slightly_Irregular,HLS,AllPub,Inside,...,No_Fence,None,0,1,2010,WD,Normal,-93.633826,42.060728,191500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,160,20000,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,No_Fence,None,0,9,2006,WD,Abnorml,-93.606842,41.987686,131000
2047,Split_or_Multilevel,Residential_Low_Density,37,7937,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,CulDSac,...,Good_Privacy,None,0,3,2006,WD,Normal,-93.604776,41.988964,142500
2048,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0,8885,Pave,No_Alley_Access,Slightly_Irregular,Low,AllPub,Inside,...,Minimum_Privacy,None,0,6,2006,WD,Normal,-93.602680,41.988314,131000
2049,Split_Foyer,Residential_Low_Density,62,10441,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,Shed,700,7,2006,WD,Normal,-93.606847,41.986510,132000


In [158]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [159]:
categorical_feature_mask = df1.dtypes==object
categorical_cols = df.columns[categorical_feature_mask].tolist()

In [160]:
categorical_cols

['MS_SubClass',
 'MS_Zoning',
 'Street',
 'Alley',
 'Lot_Shape',
 'Land_Contour',
 'Utilities',
 'Lot_Config',
 'Land_Slope',
 'Neighborhood',
 'Condition_1',
 'Condition_2',
 'Bldg_Type',
 'House_Style',
 'Overall_Qual',
 'Overall_Cond',
 'Roof_Style',
 'Roof_Matl',
 'Exterior_1st',
 'Exterior_2nd',
 'Mas_Vnr_Type',
 'Exter_Qual',
 'Exter_Cond',
 'Foundation',
 'Bsmt_Qual',
 'Bsmt_Cond',
 'Bsmt_Exposure',
 'BsmtFin_Type_1',
 'BsmtFin_Type_2',
 'Heating',
 'Heating_QC',
 'Central_Air',
 'Electrical',
 'Kitchen_Qual',
 'Functional',
 'Fireplace_Qu',
 'Garage_Type',
 'Garage_Finish',
 'Garage_Qual',
 'Garage_Cond',
 'Paved_Drive',
 'Pool_QC',
 'Fence',
 'Misc_Feature',
 'Sale_Type',
 'Sale_Condition']

In [161]:
for col in categorical_cols:
    encode = LabelEncoder()
    df1[col] = encode.fit_transform(df1[col])
    keys = encode.classes_
    values = encode.transform(encode.classes_)
    dictionary = dict(zip(keys, values))
    for index, row in df2.iterrows():
        try:
            df2[col][index] = dictionary[row[col]]
        except:
            df2[col][index] = 0
            print(1)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


1
1
1
1
1
1
1
1
1
1


In [163]:
columns = df1.columns.to_list()
columns.pop()
X_train = df1[columns].values
y_train = np.log(df1[['Sale_Price']].values)

In [164]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=.5)
clf.fit(X_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [165]:
y_test = df2[columns].values

In [166]:
y_pred = np.exp(clf.predict(y_test))

In [167]:
df3 = pd.read_csv("test_y.csv")

In [168]:
y_test = df3[['Sale_Price']].values

In [169]:
np.square((1/y_test.shape[0]) * np.sum((np.log(y_pred) - np.log(y_test))**2))

0.0007156014637328247